In [ ]:
import xgboost
import shap
import json
import warnings
warnings.filterwarnings("ignore")
shap.initjs()
import pandas as pd
df = pd.read_excel('mydata.xlsx')
df.sort_values(by=['id', 'year'], ascending=[True, True], inplace=True, ignore_index=True)
df

In [ ]:
df_new = df[~df.year.isin([2002, 2003, 2004, 2005])].reset_index().rename(columns={'index': 'original_index'})
for i in range(len(df_new)):
    df_new.HE.iloc[i] = df.HE.iloc[df_new.original_index.iloc[i]-2]
    df_new.EMSC.iloc[i] = df.EMSC.iloc[df_new.original_index.iloc[i]-4]  

In [ ]:
X = df_new.iloc[:,6:]
y = df_new['rate']

In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score
 
# 参数集定义
param_grid = {
            'max_depth': [2, 3, 4, 5, 6, 7, 8], #  基本学习器的深度： 基本学习器的数量： 要拟合的弱学习器数量，该值越大，模型越复杂，越容易过拟合树的最大深度，该值越大，模型越复杂，越容易拟合训练数据，越容易过拟合;树生长停止条件之一
            'n_estimators': [30, 50, 100, 300, 500, 1000,2000], # 基本学习器的数量： 要拟合的弱学习器数量，该值越大，模型越复杂，越容易过拟合
            'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.01, 0.02, 0.03, 0.05, 0.5],# 学习率：每个基模型的惩罚项，降低单个模型的影响，为了防止过拟合，该值越接近1越容易或拟合，越接近0精度越低
            "gamma":[0.0, 0.1, 0.2, 0.3, 0.4],#损失减少阈值： 在树的叶节点上进一步划分所需的最小损失减少，在模型训练过程中，只有损失下降的值超过该值，才会继续分裂节点，该值越小模型越复杂，越容易过拟合,树生长停止条件之一
            "reg_alpha":[0.0001,0.001, 0.01, 0.1, 1, 100],# L1正则化：L1正则化用于对叶子的个数进行惩罚,用于防止过拟合
            "reg_lambda":[0.0001,0.001, 0.01, 0.1, 1, 100],# L2正则化:L2正则化用于对叶子节点的得分进行惩罚，L1和L2正则化项共同惩罚树的复杂度,值越小模型的鲁棒性越高
            "min_child_weight": [2,3,4,5,6,7,8],
            "colsample_bytree": [0.6, 0.7, 0.8, 0.9],
            "subsample":[0.6, 0.7, 0.8, 0.9]}
# 随机搜索并打印最佳参数
gsearch1 = RandomizedSearchCV(XGBRegressor(), param_grid, cv=4)

gsearch1.fit(X, y)

print("best_score_:",gsearch1.best_params_,gsearch1.best_score_)

In [ ]:
best_model = xgboost.XGBRegressor(**gsearch1.best_params_)
model = best_model.fit(X, y)
mean_squared_error(y,model.predict(X))

In [ ]:
best_model = xgboost.XGBRegressor()
model = best_model.fit(X, y)
mean_squared_error(y,model.predict(X))
from xgboost import plot_importance
plot_importance(model)

In [ ]:
# 使用SHAP解释模型预测
model = best_model.fit(X, y)
explainer = shap.Explainer(model)
shap_values = explainer(X)
shap_values
import ipywidgets as widgets
import pyecharts.options as opts
from pyecharts.charts import Pie
# 选择样本的id和year
sample_id, year = widgets.IntSlider(value=1,min=1,max=8,step=1,description='ID:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d'),\
                  widgets.IntSlider(value=2006,min=2006,max=2021,step=1,description='YEAR:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d')
sample_id.style.handle_color = 'lightblue'
year.style.handle_color = 'lightgreen'
display(sample_id, year)
# 根据选择的id和year找到样本的index
sample_index = df_new[(df_new.id==sample_id.value) & (df_new.year==year.value)].index[0]
# 选择的样本的解释：以瀑布图形式可视化
shap.plots.waterfall(shap_values[sample_index])
# 选择的样本的解释：以力图形式可视化
shap.plots.force(shap_values[sample_index])
# 选择的样本的解释：以环形图形式可视化
x_data = list(df_new)[6:]
y_data = list(map(abs, shap_values[sample_index].values))
y_data = [i*10 for i in y_data]
print(y_data)
data_pair = [list(z) for z in zip(x_data, y_data)]
data_pair.sort(key=lambda x: x[1])
pie = (
    Pie(init_opts=opts.InitOpts())
    .add(
        series_name=f"Shap Value of Sample{sample_index} (id={sample_id.value}, year={year.value})",
        data_pair=data_pair,
        rosetype="radius",
        radius=["50%", "70%"],
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="Shap Value可视化",
            pos_left="center",
            pos_top="20",
            title_textstyle_opts=opts.TextStyleOpts(color="#2a4d69"),
        ),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%", orient="vertical"),
    )
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c}/10 ({d}%)"
        ),
        label_opts=opts.LabelOpts(),
    )
)
pie.render_notebook()

In [ ]:
df_mean = df_new.groupby('id').mean().iloc[:,5:]
df_mean

In [ ]:
combination_hill = pd.DataFrame(df_new.groupby('id').sum().iloc[:,5:].loc[[2,3,7,8]].sum()/48).T
combination_hill

In [ ]:
combination_lake = pd.DataFrame(df_new.groupby('id').sum().iloc[:,5:].loc[[1,4,5,6,9,10,11]].sum()/48).T
combination_lake

In [ ]:
# 选择的样本某年的解释：以环形图形式可视化

x_data = list(df_new)[6:]
x_data_mean = [i+" Mean" for i in x_data]
y_data = list(map(abs, combination_1_4_5_6.values.tolist()[0]))
y_data = [i for i in y_data]

data_pair = [list(z) for z in zip(x_data_mean, y_data)]
data_pair.sort(key=lambda x: x[1])
pie = (
    Pie(init_opts=opts.InitOpts())
    .add(
        series_name=f"Feature Mean of id=1, 4, 5, 6",
        data_pair=data_pair,
        rosetype="radius",
        radius=["50%", "70%"],
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title=f"湖沼型流行区干预优先级可视化",
            pos_left="center",
            pos_top="20",
            title_textstyle_opts=opts.TextStyleOpts(color="#2a4d69"),
        ),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%", orient="vertical"),
    )
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        label_opts=opts.LabelOpts(),
    )
)
pie.render_notebook()

In [ ]:
# 选择的样本某年的解释：以环形图形式可视化

x_data = list(df_new)[6:]
x_data_mean = [i+" Mean" for i in x_data]
y_data = list(map(abs, combination_2_3_7_8.values.tolist()[0]))
y_data = [i for i in y_data]

data_pair = [list(z) for z in zip(x_data_mean, y_data)]
data_pair.sort(key=lambda x: x[1])
pie = (
    Pie(init_opts=opts.InitOpts())
    .add(
        series_name=f"Feature Mean of id=2, 3, 7, 8",
        data_pair=data_pair,
        rosetype="radius",
        radius=["50%", "70%"],
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title=f"山丘型流行区干预优先级可视化",
            pos_left="center",
            pos_top="20",
            title_textstyle_opts=opts.TextStyleOpts(color="#2a4d69"),
        ),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%", orient="vertical"),
    )
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        label_opts=opts.LabelOpts(),
    )
)
pie.render_notebook()

In [ ]:
# 选择的样本某年的解释：以环形图形式可视化

x_data = list(df_new)[6:]
x_data_mean = [i+" Mean" for i in x_data]
y_data = list(map(abs, df_mean.loc[[sample_id_for_mean.value]].values.tolist()[0]))
y_data = [i for i in y_data]

data_pair = [list(z) for z in zip(x_data_mean, y_data)]
data_pair.sort(key=lambda x: x[1])
pie = (
    Pie(init_opts=opts.InitOpts())
    .add(
        series_name=f"Feature Mean of id={sample_id_for_mean.value}",
        data_pair=data_pair,
        rosetype="radius",
        radius=["60%", "80%"],
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            #title=f"id为{sample_id_for_mean.value}的特征均值可视化",
            pos_left="center",
            pos_top="20",
            title_textstyle_opts=opts.TextStyleOpts(color="#2a4d69"),
        ),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="90%", orient="vertical"),
    )
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        label_opts=opts.LabelOpts(),
    )
)
pie.render_notebook()

In [ ]:
# 所有样本的解释：以力图形式可视化
shap.force_plot(explainer.expected_value, shap_values.values, X)

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
# 取每个特征的SHAP值的绝对值的平均值作为该特征的重要性
shap.summary_plot(shap_values, X, plot_type="bar")

In [ ]:
# 特征依赖
# 横轴：Treat effects，纵轴：MSC
shap.dependence_plot("Treat", shap_values.values, X, interaction_index='MSC')

In [ ]:
from itertools import combinations
feature_list = list(df_new)[6:]
combinations_dict = {}
for n in range(3,10):
    combinations_dict[n] = []
    for i in combinations(feature_list, n):
        combinations_dict[n].append(list(i))
combinations_dict

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
mse_dict = {}
r2_dict = {}
rmse_dict = {}
mae_dict = {}
for n in combinations_dict.keys():
    mse_dict[n] = []
    r2_dict[n] = []
    rmse_dict[n] = []
    mae_dict[n] = []
    for i in combinations_dict[n]:
        X_compare = df_new[i]
        model_compare = xgboost.XGBRegressor()
        model_compare = model_compare.fit(X_compare, y)
        y_predict = model_compare.predict(X_compare)
        mse = mean_squared_error(y, y_predict)
        r2 = r2_score(y, y_predict)
        rmse = mean_squared_error(y, y_predict, squared=False)
        mae = mean_absolute_error(y, y_predict)
        mse_dict[n].append(mse)
        r2_dict[n].append(r2)
        rmse_dict[n].append(rmse)
        mae_dict[n].append(mae)
# mse_dict

In [ ]:
r2_list = []
for i in r2_dict.keys():
     for j in range(len(r2_dict[i])):
         r2_list.append([r2_dict[i][j], combinations_dict[i][j]])
mse_list = []
for i in mse_dict.keys():
     for j in range(len(mse_dict[i])):
         mse_list.append([mse_dict[i][j], combinations_dict[i][j]])
rmse_list = []
for i in rmse_dict.keys():
     for j in range(len(rmse_dict[i])):
         rmse_list.append([rmse_dict[i][j], combinations_dict[i][j]])
mae_list = []
for i in mae_dict.keys():
     for j in range(len(mae_dict[i])):
         mae_list.append([mae_dict[i][j], combinations_dict[i][j]])

In [ ]:
shap_RS = abs(shap_values.values[:, -1]).tolist()
shap_normal = np.average(abs(shap_values.values[:, :-1]), axis=1).tolist()
RS = df_new['RS'].tolist()
benefit = [i*j for i, j in zip(shap_RS, RS)]
loss = [i*j for i, j in zip(shap_normal, RS)]
RS_related = pd.DataFrame({'id': df_new['id'], 'year': df_new['year'], 'RS cost': RS, 'shap_RS': shap_RS, 
                            'shap_normal': shap_normal, 'benefit': benefit, 'loss': loss})

In [ ]:
# library
import seaborn as sns
import pandas as pd
import numpy as np
data_draw = {'cost': cost,
             'R2': pre_y,
             'label': label}